In [636]:
import gurobipy as gp
from gurobipy import *
import numpy as np
import pandas as pd

## Problem 3. Routing with Priorities

### Parameters
- $d_{i,j}$: The distance between station $i$ and station $j$.
- $E_i$: The existing number of bikes at station $i$ before rebalancing.
- $I_i = [\lambda_i - \omega_i, \lambda_i + \omega_i]$: The ideal threshold of bikes at station $i$ after rebalancing, which is represented by a predetermined idea amount $\lambda_i$ associated with a symmetric threshold of $2\omega_i$.
- $\gamma_i$: Capacity (number of docks) of bike station $i$.
- $n$: Number of stations.
- $C$: Capacity of the truck.
- $w$: Wage for driver, dollars per bike.
- $h$: Disinfecting cost, dollars per bike.
- $g$: Price of Gasoline, dollars per kilometer.
- $R$: The predicted daily revenue.
- $s_{i}^+$: A binary value that indicates whether there is a bike surplus at station $i$, i.e. $s_{i}^+ = 1$ when $E_i > \lambda_i + \omega_i$, and zero otherwise.
- $s_{i}^-$: A binary value that indicates whether there is a bike shortage at station $i$, i.e. $s_{i}^- = 1$ when $E_i < \lambda_i - \omega_i$, and zero otherwise.

In [637]:
#Data importing and preprocessing
dt_stations = pd.read_csv('/Users/Neversound/Desktop/MGSC662Proj/Bixi_filtered_hospitals.csv')
n = dt_stations.shape[0]

dt_stations['capacity'] = np.array([30 for i in range(n)])
dt_stations['priority'] = np.array(dt_stations.Hospital != 'None')


#stations = [coordinates of each station]
stations = [[np.array(dt_stations.latitude)[i], np.array(dt_stations.longitude)[i]] for i in range(n)]

In [638]:
dt_net_balance20190708 = pd.read_csv('/Users/Neversound/Desktop/MGSC662Proj/DT_net_balance2019-07-08.csv')

#Use the demand of '7/8/19 8:00' as an example
diff = np.array(dt_net_balance20190708['7/8/19 12:00'])

In [639]:
cap = np.array(dt_stations['capacity'])
ideal_level = np.floor(cap * 0.7) #ideal level is at 70%
ideal_level = ideal_level.astype(int)

#Existing number of bikes at stations, list of integers of length n
E = np.floor(cap * 0.7) + diff
E = E.astype(int)

#Manhattan Distance
#@ith WGS 1984 coordinate projection 111, 1 degree = 111 KM
d_m = [[111 * abs(i[0] - j[0]) + 111 * abs(i[1] - j[1]) for i in stations] for j in stations]

threshold = 3
I = [[ideal_level[i] - threshold, ideal_level[i] + threshold] for i in range(n)]


#Capacity of the truck
C = 40

#Binary values indicates surplus/shortage at staion i
s1 = [int(E[i] - I[i][1] > 0) for i in range(n)] #surplus
s2 = [int(I[i][0] - E[i] > 0) for i in range(n)] #shortage

#Obj parameters: wages and costs
h = 0.875
g = 0.294
R = 1000


### Variables
- $x_{i,j}$: A binary variable that indicates the relocation truck passes through station $i$ to station $j$ if $x_{i,j} = 1$, and zero otherwise.
- $c_{i}$: The current number of bikes on the truck before arriving at station $i$.
- $d_{i}$: The number of bikes the truck drops off at station $i$.
- $p_{i}$: The number of bikes the truck picks up at station $i$.

### Dummy Variables
- $z_i$: Non-decreasing variables to ensure continuity of the route, for $i \in \{1 \dots n\}$.


### Other Notations
- $B$: The total number of bikes that are relocated.
- $D$: The total distance that the truck has traveled.
- $V$: The total number of stations visited (for record tracking purpose ONLY).

In [640]:
#Priority parameter
priority = list(dt_stations['priority'])

In [641]:
#Profit Maximization Model
prob = Model('BIXI')

x = prob.addVars(n, n, vtype='B', name = 'x') #BINAR"Y: x_{i,j}
c = prob.addVars(n,lb = 0, vtype='I', name = 'c') #INTEGER: c_i 
d = prob.addVars(n,lb = 0, vtype='I', name = 'd') #INTEGER: d_i
p = prob.addVars(n,lb = 0, vtype='I', name = 'p') #INTEGER: p_i
z = prob.addVars(n,lb = 0, name = 'z')

B = prob.addVar(lb = 0, vtype='I', name = 'B') #INTEGER: total number of bikes rebalanced
D = prob.addVar(lb = 0, name = 'D') #INTEGER: total distance travelled by truck

#Variable to check accuracy
V = prob.addVar(lb = 0, vtype='I', name = 'V') #INTEGER, total number of stations visited
d_sum = prob.addVar(lb = 0, vtype='I', name = 'd_sum') #INTEGER, total number of bikes dropped off
p_sum = prob.addVar(lb = 0, vtype='I', name = 'p_sum') #INTEGER, total number of bikes picked up

### Assumptions
- Routing
  - 1. Each station is visited at most once.
  - 2. The truck starts at station 0 and ends at station 0.
  - 3. The distance will be calcuated using Manhattan distance.
- Rebalancing
  - 1. The truck arrives at station 0 with 20 bikes at the beginning of each working hour.
  - 2. The operational cost has three components, the wage for the worker, gas cost and disinfecting cost.

### Variations
- fixed subpath on bike stations located near hospitals, extra cost for disinfecting during COVID-19
- Nonlinear objective

### Objective
$$\text{Minimize } \big( h * B + g * D + w\big)$$

In [642]:
obj = h * B + g * D + 20.52
prob.setObjective(obj, GRB.MINIMIZE)

### Constraints - Routing

$$\begin{align}
\sum_i x_{ij} & \le 1 \hspace{0.5cm} i = \{2 \dots n\} \tag{Routing-1.1}\\
\sum_j x_{ij} & \le 1 \hspace{0.5cm} j = \{2 \dots n\} \tag{Routing-1.2}\\
\sum_i x_{ij} & = \sum_i x_{ji} \hspace{0.5cm} \tag{Routing-1.3}\\
\sum_i x_{ii} & = 0 \hspace{0.5cm} \tag{Routing-1.4}\\
\sum_i x_{i1} & = 1 \tag{Routing-2.1}\\
\sum_j x_{1j} & = 1 \tag{Routing-2.2}\\
\sum_{i,j} x_{ij} d_{ij} & = D  \tag{Total distance}\\
z_j & \ge z_i + 1 - 1996(1 - x_{ij}) \tag{Continuity of route}\\
\end{align}$$

In [643]:
visiting_arr = np.array(s1) + np.array(s2)
priority_list = [i for i in range(n) if visiting_arr[i] == 1 and priority[i] == 1]
regular = [num for num in range(n) if num != 0 and num not in priority_list]

In [644]:
#prob.addConstr(quicksum(x[i,j] for i in priority for j in regular) <= 1)

In [645]:
#---Routing---

#Routing-1.1
for j in range(n):
    prob.addConstr(quicksum(x[i,j] for i in range(n)) <= 1)
    
#Routing-1.2    
for i in range(n):
    prob.addConstr(quicksum(x[i,j] for j in range(n)) <= 1)

#Routing-1.3
for i in range(n):
    prob.addConstr(quicksum(x[i,j] for j in range(n)) == quicksum(x[j,i] for j in range(n))) 

#Routing-1.4
for i in range(1,n):
    prob.addConstr(x[i,i] == 0)    

#Total Distance
prob.addConstr(quicksum(x[i,j] * d_m[i][j] for i in range(n) for j in range(n)) == D)

#Route Continuity
#Avoid sub trips by implementing a non-decreasing variable z
#When the truck travels from i to j, z[j] > z[i]
for i in range(n):
    for j in range(1,n):
        prob.addConstr(z[j] >= z[i] + 1 - 199996 * (1 - x[i,j]))

### Constraints - Rebalancing

$$\begin{align}
c_i & \le C \tag{Capacity of the truck}\\
s_i^- d_i + s_i^+ p_i & = 0 \tag{Pick up XOR Drop off at any station}\\
p_i & \le E_i \tag{Pick up within ideal threshold}\\
\sum_i d_i & \le \sum_i p_i \tag{Drop off LESS THAN pick up}\\
\sum_i d_i + p_i & = B \tag{Total amount of bikes rebalanced}\\
d_j + p_j & \ge \sum_i x_{ij} \tag{A station is visted when necessary}\\
c_j + d_j + p_j + \alpha_j & \le 1996\sum_i x_{ij} \tag{A station is visted when necessary}\\
c_j & \ge c_i - d_i + p_i - C(1 - x_{ij}) \hspace{0.5cm} \tag{Continuity of truck inventory - 1}\\
c_j & \le c_i - d_i + p_i - C(1 - x_{ij}) \hspace{0.5cm} \tag{Continuity of truck inventory - 2}\\
\end{align}$$

In [646]:
#---Rebalancing---     
        
#Truck arrives at station 1 with 20 bikes
prob.addConstr(c[0] == 20)


for i in range(1,n):
    #Pick up XOR Drop off at any station
    prob.addConstr(s1[i] * d[i] + s2[i] * p[i] == 0)

    #Capacity of the truck
    prob.addConstr(c[i] <= C)
    
    #pick up within ideal threshold
    prob.addConstr(p[i] <= E[i])
    prob.addConstr(p[i] >= E[i] - I[i][1])
    
    #drop off within ideal threshold
    prob.addConstr(d[i] <= cap[i] - E[i])
    prob.addConstr(d[i] >= (I[i][0] - E[i])) #nonbinding when surplus  
    
#Total amount of bikes rebalanced
prob.addConstr(quicksum(p[i] + d[i] for i in range(n)) == B)

#A station is visted when necessary (synergy with the route)
for i in range(1,n):
    prob.addConstr(d[i] + p[i] >= quicksum(x[j,i] for j in range(n)))
    #when station i is not visited LHS = RHS = 0
    #when station i is visited 
    prob.addConstr(c[i] + d[i] + p[i] + z[i] <= 199996 * quicksum(x[j,i] for j in range(n)))
                   

#Continuity of truck inventory
for i in range(n):
    for j in range(1,n):
        prob.addConstr(c[j] <= c[i] - d[i] + p[i] + C * (1 - x[i,j]))
        prob.addConstr(c[j] >= c[i] - d[i] + p[i] - C * (1 - x[i,j]))

In [647]:
#---Variables for record tracking---           
prob.addConstr(quicksum(x[i,j] for i in range(n) for j in range(n)) == V)    
prob.addConstr(quicksum(d[i] for i in range(n)) == d_sum)
prob.addConstr(quicksum(p[i] for i in range(n)) == p_sum)

<gurobi.Constr *Awaiting Model Update*>

In [648]:
prob.optimize()
prob.printAttr('X')

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10611 rows, 3601 columns and 70187 nonzeros
Model fingerprint: 0x855a2c2c
Variable types: 59 continuous, 3542 integer (3364 binary)
Coefficient statistics:
  Matrix range     [5e-02, 2e+05]
  Objective range  [3e-01, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+05]
Presolve removed 641 rows and 21 columns
Presolve time: 0.15s
Presolved: 9970 rows, 3580 columns, 48417 nonzeros
Variable types: 57 continuous, 3523 integer (3357 binary)
Found heuristic solution: objective 44.6562214

Root relaxation: objective 3.296426e+01, 245 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   32.96426    0   12   44.65622   32.96426  26.2%     -    0s
H    0     0                      42.42

In [649]:
print('Operation cost for July 8th 12:00 - 13:00: ' + str(round(obj.getValue(),2)))
print('The driver travelled {} stations and rebalanced {} bikes (picked up {} bikes and dropped off {} bikes).'.format(V.x, B.x, p_sum.x, d_sum.x))

Operation cost for July 8th 12:00 - 13:00: 33.94
The driver travelled 8.0 stations and rebalanced 13.0 bikes (picked up 9.0 bikes and dropped off 4.0 bikes).


### Operation cost for regular routing on July 8th 12:00 - 13:00: 33.94
### The driver travelled 8.0 stations and rebalanced 13.0 bikes (picked up 9.0 bikes and dropped off 4.0 bikes).
### The route is 0,11,14,28,32,3,6,4,0

In [651]:
priority_list

[0, 3, 4, 28]

### Note that station 0, 3, 4 and 28 are stations near hospitals that needs rebalancing, now we add the priority constraint.

In [652]:
#Profit Maximization Model
prob = Model('BIXI')

x = prob.addVars(n, n, vtype='B', name = 'x') #BINAR"Y: x_{i,j}
c = prob.addVars(n,lb = 0, vtype='I', name = 'c') #INTEGER: c_i 
d = prob.addVars(n,lb = 0, vtype='I', name = 'd') #INTEGER: d_i
p = prob.addVars(n,lb = 0, vtype='I', name = 'p') #INTEGER: p_i
z = prob.addVars(n,lb = 0, vtype='I', name = 'z')

B = prob.addVar(lb = 0, vtype='I', name = 'B') #INTEGER: total number of bikes rebalanced
D = prob.addVar(lb = 0, name = 'D') #INTEGER: total distance travelled by truck

#Variable to check accuracy
V = prob.addVar(lb = 0, vtype='I', name = 'V') #INTEGER, total number of stations visited
d_sum = prob.addVar(lb = 0, vtype='I', name = 'd_sum') #INTEGER, total number of bikes dropped off
p_sum = prob.addVar(lb = 0, vtype='I', name = 'p_sum') #INTEGER, total number of bikes picked up

In [653]:
obj = h * B + g * D + 20.52 + quicksum(z[i] for i in priority_list)
prob.setObjective(obj, GRB.MINIMIZE)

In [654]:
#Priority constraints
prob.addConstr(quicksum(x[i,j] for i in priority_list for j in regular) <= 1)

prob.addConstr(quicksum(x[0,i] for i in priority_list) <= 1)

<gurobi.Constr *Awaiting Model Update*>

In [655]:
#---Routing---

#Routing-1.1
for j in range(n):
    prob.addConstr(quicksum(x[i,j] for i in range(n)) <= 1)
    
#Routing-1.2    
for i in range(n):
    prob.addConstr(quicksum(x[i,j] for j in range(n)) <= 1)

#Routing-1.3
for i in range(n):
    prob.addConstr(quicksum(x[i,j] for j in range(n)) == quicksum(x[j,i] for j in range(n))) 

#Routing-1.4
for i in range(1,n):
    prob.addConstr(x[i,i] == 0)    

#Total Distance
prob.addConstr(quicksum(x[i,j] * d_m[i][j] for i in range(n) for j in range(n)) == D)

#Route Continuity
#Avoid sub trips by implementing a non-decreasing variable z
#When the truck travels from i to j, z[j] > z[i]
for i in range(n):
    for j in range(1,n):
        prob.addConstr(z[j] >= z[i] + 1 - 199996 * (1 - x[i,j]))

In [656]:
#---Rebalancing---     
        
#Truck arrives at station 1 with 20 bikes
prob.addConstr(c[0] == 20)


for i in range(1,n):
    #Pick up XOR Drop off at any station
    prob.addConstr(s1[i] * d[i] + s2[i] * p[i] == 0)

    #Capacity of the truck
    prob.addConstr(c[i] <= C)
    
    #pick up within ideal threshold
    prob.addConstr(p[i] <= E[i])
    prob.addConstr(p[i] >= E[i] - I[i][1])
    
    #drop off within ideal threshold
    prob.addConstr(d[i] <= cap[i] - E[i])
    prob.addConstr(d[i] >= (I[i][0] - E[i])) #nonbinding when surplus  
    
#Total amount of bikes rebalanced
prob.addConstr(quicksum(p[i] + d[i] for i in range(n)) == B)

#A station is visted when necessary (synergy with the route)
for i in range(1,n):
    prob.addConstr(d[i] + p[i] >= quicksum(x[j,i] for j in range(n)))
    #when station i is not visited LHS = RHS = 0
    #when station i is visited 
    prob.addConstr(c[i] + d[i] + p[i] + z[i] <= 199996 * quicksum(x[j,i] for j in range(n)))
                   

#Continuity of truck inventory
for i in range(n):
    for j in range(1,n):
        prob.addConstr(c[j] <= c[i] - d[i] + p[i] + C * (1 - x[i,j]))
        prob.addConstr(c[j] >= c[i] - d[i] + p[i] - C * (1 - x[i,j]))

In [657]:
#---Variables for record tracking---           
prob.addConstr(quicksum(x[i,j] for i in range(n) for j in range(n)) == V)    
prob.addConstr(quicksum(d[i] for i in range(n)) == d_sum)
prob.addConstr(quicksum(p[i] for i in range(n)) == p_sum)

<gurobi.Constr *Awaiting Model Update*>

In [658]:
prob.optimize()
prob.printAttr('X')

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10613 rows, 3601 columns and 70407 nonzeros
Model fingerprint: 0xbeb6a000
Variable types: 1 continuous, 3600 integer (3364 binary)
Coefficient statistics:
  Matrix range     [5e-02, 2e+05]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+05]
Presolve removed 642 rows and 21 columns
Presolve time: 0.16s
Presolved: 9971 rows, 3580 columns, 48633 nonzeros
Variable types: 0 continuous, 3580 integer (3357 binary)

Root relaxation: objective 3.333779e+01, 178 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   33.33779    0   18          -   33.33779      -     -    0s
H    0     0                      80.0119775   33.33779  58.3%     -    0s
H    0     0 

In [660]:
print('Operation with priorities cost for July 8th 12:00 - 13:00: ' + str(round(obj.getValue(),2)))
print('The driver travelled {} stations and rebalanced {} bikes (picked up {} bikes and dropped off {} bikes).'.format(V.x, B.x, p_sum.x, d_sum.x))

Operation with priorities cost for July 8th 12:00 - 13:00: 40.14
The driver travelled 8.0 stations and rebalanced 13.0 bikes (picked up 9.0 bikes and dropped off 4.0 bikes).


In [611]:
priority_list

[0, 3, 4, 28]

# Before
### Running time < 1s
### Operation cost for regular routing on July 8th 12:00 - 13:00: 33.94
### The driver travelled 8.0 stations and rebalanced 13.0 bikes (picked up 9.0 bikes and dropped off 4.0 bikes).
### The route is 0,11,14,28,32,3,6,4,0

# After
### Running time: 36.84s
### Operation cost for regular routing on July 8th 12:00 - 13:00: 40.14
### The driver travelled 8.0 stations and rebalanced 13.0 bikes (picked up 9.0 bikes and dropped off 4.0 bikes).
### The route is 0,4,3,28,32,6,14,11,0
### Note that all stations are visited, but all the prioritized stations are visited first.